# Allowed libraries
- Tensorflow (compatible with 1.12.x)
- Numpy
- Sklearn
- nltk
- Maplotlib
- gensim
- All the standard libraries
 

https://medium.com/the-artificial-impostor/nlp-four-ways-to-tokenize-chinese-documents-f349eb6ba3c3

https://stanfordnlp.github.io/CoreNLP/download.html

In [1]:
import os
from sklearn.preprocessing import OneHotEncoder

In [2]:
class Make_Feed(object):
    '''makes feed files of combined unigrams and bigrams'''
    def __init__(self):
        pass

In [3]:
datasets = {"training":'../icwb2-data/training',
             "dev":'../icwb2-data/gold',
             "testing":'../icwb2-data/testing'}

In [4]:
def get_file_names(path, type_='LabelFile'):
    x = []
    for i in os.listdir(path):
        if os.path.splitext(i)[0].split("_")[-1] == type_:
            x.append(os.path.join(path, i))
    return x

Label_files = get_file_names(path = datasets['training'], type_ = 'LabelFile')
Input_files = get_file_names(path = datasets['training'], type_ = 'InputFile')

In [8]:
Input_files

['../icwb2-data/training/msr_training_simplified_InputFile.utf8',
 '../icwb2-data/training/cityu_training_simplified_InputFile.utf8',
 '../icwb2-data/training/as_training_simplified_InputFile.utf8',
 '../icwb2-data/training/pku_training_simplified_InputFile.utf8']

In [6]:
from typing import Tuple, List, Dict

def split_into_grams(sentence: str, type_ = 'uni_grams') -> List[str]:
    """
    :param sentence Sentence as str
    :type_: uni_grams or _bigrams
    :return bigrams List of unigrams or bigrams
    """
    n = 1 if type_ == 'uni_grams' else 2
    grams = []
    for i in range(len(sentence)-1):
        gram = sentence[i:i+n]
        grams.append(gram)
    return grams


In [9]:
big_line = ''

with open(Input_files[-1], 'r', encoding ='utf8') as f1:
    for line in f1:
        big_line+=line.rstrip()
        
final = split_into_grams(big_line, type_ = 'bi_grams') + split_into_grams(big_line, type_ = 'uni_grams')


In [ ]:
file_ = "../icwb2-data/training/pku_training_simplified_InputFile_FEED.utf8"
with open(file_, 'w') as t:
    for item in final:
        t.write("%s\n" % item)

In [ ]:
final_1 = []
with open(file_, 'r', encoding ='utf8') as f1:
    for line in f1:
        final_1.append(line.rstrip())

In [10]:
vocab = set(final)

In [11]:
word_to_index = {value:key for key,value in enumerate(vocab)}
word_to_index['UNK'] = 0

In [27]:
import numpy as np

# creating feature vectors

In [13]:
features_vectors = []
with open(Input_files[-1], 'r', encoding ='utf8') as f1:
    for line in f1:
        l = line.rstrip()
        grams = split_into_grams(l, 'uni_grams') + split_into_grams(l,'bi_grams')
        features_vectors.append([word_to_index[i] for i in grams])

In [46]:
def OHE(label):
    label = label.reshape(-1, 1)
    enc = OneHotEncoder()
    enc.fit(label)
    return enc.transform(label).toarray()
BIES = {'B' : 0, 'I' : 1, 'E' : 2, 'S' : 3}

In [53]:


with open(Label_files[-1], 'r', encoding ='utf8') as f1:
    count = 0
    for line in f1:
        count+=1
        if count==10:
            break
        l = line.rstrip()
        label = np.array([BIES[i] for i in l])
        print(OHE(label).shape)

(43, 4)
(21, 4)
(18, 4)
(86, 4)
(27, 3)
(156, 4)
(280, 4)
(334, 4)
(303, 4)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and sil

In [45]:
OHE(label)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],


In [ ]:
o = np.zeros((len(label), len(label)))
for i in range(o.shape[0]):
    o[i,i]=1

In [ ]:
o

- original file $\rightarrow$ simplified Chinese
- Input file $\rightarrow$ used to feed Bi-LSTM model
- Label file $\rightarrow$ used to test the predictions

TO DO: probably need a decoder

In [ ]:
from nltk.tokenize.stanford_segmenter import StanfordSegmenter

In [ ]:
from nltk.corpus import stopwords
eng_stword = stopwords.words("german")
eng_stword

In [ ]:
>>> from nltk.tokenize.stanford_segmenter import StanfordSegmenter
>>> segmenter = StanfordSegmenter(path_to_jar=”stanford-segmenter-3.4.1.jar”, path_to_sihan_corpora_dict=”./data”, path_to_model=”./data/pku.gz”, path_to_dict=”./data/dict-chris6.ser.gz”)
>>> sentence = u”这是斯坦福中文分词器测试”
>>> segmenter.segment(sentence)
>>> u’\u8fd9 \u662f \u65af\u5766\u798f \u4e2d\u6587 \u5206\u8bcd\u5668 \u6d4b\u8bd5\n’
>>> segmenter.segment_file(“test.simp.utf8”)
>>> u’\u9762\u5bf9 \u65b0 \u4e16\u7eaa \uff0c \u4e16\u754c \u5404\u56fd .
>>> outfile = open(‘outfile’, ‘w’)
>>> result = segmenter.segment(sentence)
>>> outfile.write(result.encode(‘UTF-8′))
>>> outfile.close()

In [ ]:
sentence = '这是斯坦福中文分词器测试'

In [ ]:
>>> from nltk.tokenize.stanford_segmenter import StanfordSegmenter

In [ ]:
#segmenter = StanfordSegmenter()
p = '../resources/stanford-chinese-corenlp-2018-10-05-models.jar'

In [ ]:
segmenter = StanfordSegmenter(path_to_jar=p)

In [ ]:
segmenter.segment(sentence)